# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 00 : initialisation de l'environnement et ingestion des données

---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Date de création :** 26 Décembre 2025
* **Dernière mise à jour :** 29 Décembre 2025
* **Dépendances notebooks**: Aucune

### Description
Ce notebook établit les fondations du projet. Il assure la mise en place de l'infrastructure et la configuration des outils de gestion . L'objectif est de garantir une base reproductible avant d'entamer l'audit de qualité.

### Objectifs principaux
1. Initialiser le système de gestion de configuration via Hydra.
2. Déployer l'arborescence des répertoires du projet.
3. Récupérer et charger le jeu de données source (Immuable).
4. Établir un premier diagnostic structurel des données chargées.

### Dépendances critiques
* `hydra` : Gestion de la configuration.
* `src.utils` & `src.data` : Modules internes de support.

### LIVRABLES
1. Environnement technique initialisé (Hydra, Dossiers).
2. Dataset brut dans `data/raw/`.
3. Dictionnaire de données initial (`reports/data_dictionary.md`).
4. Note de synthèse structurelle.
5. Notebook propre

---

# Table des Matières
- [Section 0 : Importation des packages ](#section-1--configuration)
- [Section 1 : Configuration et initialisation ](#section-1--configuration)
- [Section 2 : Contextualisation & provenance](#section-2--contextualisation--provenance)
- [Section 3 : Chargement et inspection structurelle](#section-3--ingestion-et-premier-contact)
- [Section 4 : Typologie des variables](#section-4--typologie-et-cartographie-des-variables)
- [Section 5 : Focus sur la variable cible](#section-5--analyse-de-la-variable-cible)
- [Section 6 : Identification des redondances apparentes](#section-6--étude-des-redondances-et-unités)
- [Section 7 :Premières incohérences détectées](#section-7--détection-des-signaux-dalerte-immédiats)
- [Section 8 : Synthèse et génération du dictionnaire](#section-8--synthèse-et-génération-du-dictionnaire)
---

# Section 0 : Importation des packages

In [ ]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger
 


# Section 1:Configuration et initialisation

In [8]:
# Chargement de la configuration principale (Hydra)
cfg = load_config()

# Initialisation du logger
setup_eda_logger(cfg)
logger = logging.getLogger(__name__)

# Création des dossiers nécessaires à l'exécution (raw, interim, processed, reports, etc.)
create_directories(cfg)


2025-12-29 20:48:49,087 - utils.config_loader - INFO - Configuration 'config' chargée (project_root=C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2)
2025-12-29 20:48:49,090 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2025-12-29 20:48:49,091 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2025-12-29 20:48:49,092 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2025-12-29 20:48:49,094 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2025-12-29 20:48:49,096 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports


# Section 2 : Ingestion et Chargement des Données Brutes

In [ ]:
# Chargement des données brutes
df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")

# Premier aperçu
df_raw.sample(5)


2025-12-29 20:49:26,419 - data.load_data - INFO - Fichier local détecté : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw\2016_Building_Energy_Benchmarking.csv
2025-12-29 20:49:26,467 - data.load_data - INFO - ✓ DataFrame chargé : 3376 lignes, 46 colonnes
2025-12-29 20:49:26,473 - __main__ - INFO - Dataset chargé (3376 lignes, 46 colonnes)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
966,20377,2016,NonResidential,Senior Care Community,Washington Care Center,2821 S Walden St,Seattle,WA,98144.0,1624049040,...,973339.8125,3321035.0,0.000000,0.0,False,NaN,Compliant,NaN,23.15,0.30
1434,21792,2016,NonResidential,Warehouse,OTM 162,5964 6th Ave. S,Seattle,WA,98108.0,2024049059,...,239011.5938,815508.0,3929.000000,392900.0,False,NaN,Compliant,NaN,26.55,1.02
633,857,2016,NonResidential,Warehouse,3480 W Marginal Way SW,3480 W Marginal Way SW,Seattle,WA,98053.0,7666705088,...,277903.5000,948207.0,1215.430054,121543.0,False,NaN,Compliant,NaN,13.07,0.09
1111,20890,2016,NonResidential,K-12 School,ASB School,6220 32nd Ave. NE,Seattle,WA,98115.0,1868903240,...,142531.0938,486316.0,12191.990230,1219199.0,False,NaN,Compliant,NaN,68.14,2.94
855,20021,2016,Multifamily LR (1-4),Low-Rise Multifamily,Century House Apartments,1711 23rd Ave. S.,Seattle,WA,98144.0,0924049006,...,692602.8125,2363161.0,0.000000,0.0,False,NaN,Compliant,NaN,16.47,0.28
